# An Introduction to wavelets 

by Charles Dossal

In [ ]:
import numpy as np
import scipy as scp
import pylab as pyl
import matplotlib.pyplot as plt
import pywt
import scipy.io as sio
import pandas as pd
import holoviews as hv
hv.extension('bokeh')
import param
import panel as pn
import requests
from io import BytesIO
from PIL import Image
import shutil
from scipy import fftpack
from scipy.io.wavfile import read
from IPython.display import Audio
from urllib.request import urlopen
import io
from scipy.io.wavfile import read
import scipy.io as sio

In [ ]:
local=0
def chargeData(name):
    if local:
        if name=='Lenna':
            res=np.array(Image.open("./Archive/img/Lenna.jpg")).astype(float)
        if name=='Canaletto':
            res=np.array(Image.open("./Archive/img/Canaletto.jpeg")).astype(float)
        if name=='Minotaure':
            res=np.array(Image.open("./Archive/img/MinotaureBruite.jpeg")).astype(float)   
        if name=='Cartoon':
            res=np.array(Image.open("./Archive/img/Cartoon.jpg")).astype(float) 
        if name=='Croco1':
            res=np.array(Image.open("CrocoOriginal.png")).astype(float) 
        if name=='Croco2':
            res=np.array(Image.open("CrocoDebruit.png")).astype(float) 
        if name=='Monument':
            res=np.array(Image.open("Monument.png")).astype(float) 
        if name=='Monument2':
            res=np.array(Image.open("Ondlev1.png")).astype(float) 
        if name=='Monument3':
            res=np.array(Image.open("Ondlev2.png")).astype(float) 
        if name=='Monument4':
            res=np.array(Image.open("Ondlev3.png")).astype(float) 
        if name=='avion':
            res=np.array(Image.open("TestImageOndelettes.png")).astype(float) 
        if name=='avion2':
            res=np.array(Image.open("TestImageOndelettesNette.png")).astype(float) 
        if name=='LennaMasked':
            res=np.array(Image.open("LennaMasked50.png")).astype(float) 
        if name=='LennaInpainted':
            res=np.array(Image.open("LennaInpainted50.png")).astype(float) 
    else:
        if name=='Lenna':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/Lenna.jpg'        
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
        if name=='Canaletto':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/Canaletto.jpeg'
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
        if name=='Minotaure':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/MinotaureBruite.jpeg'
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
        if name=='Cartoon':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/Cartoon.jpg'        
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
        if name=='Monument':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/Monument.png'        
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
        if name=='Monument2':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/Ondlev1.png'        
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
        if name=='Monument3':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/Ondlev2.png'        
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
        if name=='Monument4':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/Ondlev3.png'        
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
        if name=='Croco1':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/CrocoOriginal.png'        
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
        if name=='Croco2':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/CrocoDebruit.png'        
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
        if name=='avion':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/TestImageOndelettes.png'        
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
        if name=='avion2':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/TestImageOndelettesNette.png'        
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
        if name=='LennaMasked':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/LennaMasked50.png'        
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
        if name=='LennaInpainted':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/LennaInpainted50.png'        
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
    return res
options1=dict(width=400,height=400,xaxis=None,yaxis=None,toolbar=None)
options2=dict(width=800,height=400,xaxis=None,yaxis=None,toolbar=None)

In [ ]:

if local:
    S3=np.load('Piece.npy')
    S4=np.load('Blocks.npy')   
else:
    url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/Blocks.npy'
    response = requests.get(url, stream=True)
    with open('blocks.npy', 'wb') as fin:
        shutil.copyfileobj(response.raw, fin)
    S4=res=np.load('blocks.npy')
    url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/Piece.npy'
    response = requests.get(url, stream=True)
    with open('piece.npy', 'wb') as fin:
        shutil.copyfileobj(response.raw, fin)
    S3=res=np.load('piece.npy')
#pn.Row(hv.Curve(S3).opts(width=450),hv.Curve(S4).opts(width=450))

## The power of the Fourier Transform and its limits.

The Fourier analysis allow the decomposition of a function, a vector, an image as a linear combination of complex exponential. If $f\in L^2(\mathbb{R})$, this combination is infinite and if $f$ is a vector of $\mathbb{R}^N$, the combination contains $N$ terms. Each Fourier coefficient $c_n(f)$ of $\hat f[k]$ can be seen as a coefficient in this decomposition. The power of the Fourier analysis is that most functions and all vectors can be written as such linear combinations. Hence, most functions can be described by its sequence of Fourier coefficients. 

Unfortunalty linear combinations with a huge number of terms are practically useless. In many practical problems, we would prefer an approximation of $f$ with a small number of coefficients, even if the price to pay is an approximation error. It turns out that some functions can be well approximated using a Fourier Transform. More precisely, the more a function is regular, i.e. the more continuous derivative it has, the best the approximations are. This qualitative remark can be stated theorically with the following Theorem

\begin{theorem}
If $f$ has $p$ continuous derivative, the linear approximation error in the Fourier domain decays at least as fast as $\frac{1}{N^{2p+1}}$, i.e. it exists $C>0$ 
\begin{equation}
\Vert f-\sum_{k=-N}^Nc_{k}(f)e^{ik\cdot}\Vert^2\leqslant \frac{C}{N^{2p+1}}
\end{equation}
\end{theorem}
The decay of the error as a function of the number of terms in the Fourier approximation highly depends on the number of continuous derivative $f$ have. 
Indeed if $f$ is infinitly derivable, the Fourier approximations fits very well and there is no need to consider further analysis. But if $f$ has a single discontinuity, the approximation abilities of the Fourier transform vanishe and many terms are necessary to provide a fair approximation of $f$. 

Indeed what is true for functions is also true for vectors : if a vector $v$ is a discrtisation of a smooth function $f$, good Fourier approximations of $v$ can be found with a small number of coefficients. If $v$ can be seen as a discretization of discontinuous function $f$, more coefficients are needed to provide a fair approximation of $v$. 

This observation leads ingeneers in the early 80s to divide the images into squares of $8\times 8$ pixels before the computation of a Fourier analysis to define the Jpeg standart. On most small $8\times 8$ images, there is no discontinuity of the luminance level and the Fourier tranform may provide an efficient representation of images.

A global Fourier transform over the whole image would have fail to produce an efficient compression algorithm because of the numerous discontinuities of the luminance at the border of each object. 

Indeed, natural images can't be considered as smooth functions but most of them can be approximated by piecewise smooth. The question arising is : Is there any way to provide an efficient representation of piecewise smooth functions ? The answer is : Yes, using wavelets.

In [ ]:
S4=res=np.load('blocks.npy')
S3=res=np.load('piece.npy')
Fe=44100
SonGuitare="Guitare.wav"
gui=read(SonGuitare)
guit=gui[1]/2**14
Audio(guit, rate=Fe)


In [ ]:
Saxo=sio.loadmat('Saxo.mat')
Saxo1=Saxo['S']
Saxo2=Saxo['S1']
temp=np.zeros(913)
for k in range(0,913):
    temp[k]=Saxo2[k]
Saxo2=temp
N1=int(len(Saxo1)/3)
temp=np.zeros(N1)
for k in range(0,N1):
    temp[k]=Saxo1[k]
sax=temp
d2=64000+29
Ext=sax[d2:d2+1462]
d2=64000
Ext2=guit[d2:d2+1500]
Audio(sax, rate=Fe)

In [ ]:
def SommePartielle(f,m):
    Nf=np.size(f)
    temp=fftpack.fft(f)
    for k in range(m+1,Nf-m):
        temp[k]=0
    temp2=fftpack.ifft(temp)
    S=np.real(temp2)
    return S
signauxRef2= {"Piece" : S3,"Blocks" : S4,"Guitare" : Ext2,"Saxo" : Ext}
def ApproxFourier2(s,n):
    N=np.shape(s)[0]
    fs=fftpack.fft(s)
    afs=np.abs(fs)
    temp=np.argsort(afs)
    fsrec=fs*0
    fig=hv.Curve(s)
    for k in np.arange(1,2*n+1):
        fsrec[temp[N-k]]=fs[temp[N-k]]
        if np.mod(k+1,2)<1:
            ftemp=0*fs
            ftemp[temp[N-k]]=fs[temp[N-k]]
            sin=2*np.real(fftpack.ifft(ftemp))
            if k<2:
                fig=hv.Curve(sin)
            else:
                fig=fig*hv.Curve(sin)
    fapprox=np.real(fftpack.ifft(fsrec))
    return fapprox,fsrec,fig
class ApproxFourier(param.Parameterized):
    signal = param.ObjectSelector(default="Guitare",objects=signauxRef2.keys())
    N = param.Integer(20,bounds=(1,200))
    def view(self):
        s=signauxRef2[self.signal]
        options = dict(width=800,height=250,toolbar=None,xaxis=None,yaxis=None)
        fsrec=SommePartielle(s,self.N)
        strN="%2.0f" % self.N
        text=hv.Text(50,150,'Nombre de coefficients= '+strN+' sur 750').opts(**options).opts(fontsize=30)
        return pn.Column(hv.Curve(s).opts(**options)*
                         hv.Curve(fsrec).opts(color='red').opts(**options))

In [ ]:
Af= ApproxFourier()
pn.Column(Af.param,Af.view)

# Wavelet analysis.

## Lazy Wavelet schemes.

Wavelets are functions that are able to provide efficient representations of piecewise smooth functions. In this course we will focus on orthogonal wavelet bases but the wavelet theory is indeed much extensive, including for instance the continuous wavelet transform. 

There exists many wavelet bases and on this course we will start with Haar wavelets and then we will study other bazses such like Daubechies Wavelets and Coifmann wavelets. It is impossible to cite all mathematician who contribute to wavelets but we can mention 

Ingrid Daubechies : https://en.wikipedia.org/wiki/Ingrid_Daubechies

Ronald Coifmann : https://fr.wikipedia.org/wiki/Ronald_Coifman

who gave their name to usefull wavelet bases and 

Yves Meyer : https://fr.wikipedia.org/wiki/Yves_Meyer

Stéphane Mallat : https://fr.wikipedia.org/wiki/Stéphane_Mallat

who proved together the main Theorem of the wavelet theory giving a sufficient and necessary condition ensuring a function can provide an orthognal wavelet basis.
This four mathematicians are already active. The two last were my teachers and Stephane Mallat who is now Professor at "The College de France" where he teaches Data Sciences and is a member of the "Académie des sciences" was my PhD advisor.


In [ ]:
def Ond(N,qmf,scale,k):
    Lmax=pywt.dwt_max_level(N,pywt.Wavelet(qmf).dec_len)
    L=min(scale,Lmax)
    Ne=int(np.floor(N/2**L))
    #k=int(np.floor(Ne*pos))
    temp=np.zeros(N)
    WT= pywt.wavedecn(temp, qmf, mode='per', level=L)
    arr, coeff_slices = pywt.coeffs_to_array(WT)
    temp[k+Ne]=1
    coeffs_from_arr = pywt.array_to_coeffs(temp, coeff_slices)
    Psi=pywt.waverecn(coeffs_from_arr,qmf,mode='per')
    return Psi
def FoncEchel(N,qmf,scale,k):
    Lmax=pywt.dwt_max_level(N,pywt.Wavelet(qmf).dec_len)
    L=min(scale,Lmax)
    Ne=int(np.floor(N/2**L))
    #k=int(np.floor(Ne*pos))
    temp=np.zeros(N)
    WT= pywt.wavedecn(temp, qmf, mode='per', level=L)
    arr, coeff_slices = pywt.coeffs_to_array(WT)
    temp[k]=1
    coeffs_from_arr = pywt.array_to_coeffs(temp, coeff_slices)
    Phi=pywt.waverecn(coeffs_from_arr,qmf,mode='per')
    return Phi
def VisuFoncEchel(N,qmf,scale,pos):
    Lmax=pywt.dwt_max_level(N,pywt.Wavelet(qmf).dec_len)
    L=min(scale,Lmax)
    Ne=int(np.floor(N/2**L))
    k=int(np.floor(Ne*pos))
    temp=np.zeros(N)
    WT= pywt.wavedecn(temp, qmf, mode='per', level=L)
    arr, coeff_slices = pywt.coeffs_to_array(WT)
    temp[k]=1
    coeffs_from_arr = pywt.array_to_coeffs(temp, coeff_slices)
    Phi=pywt.waverecn(coeffs_from_arr,qmf,mode='per')
    return hv.Curve(Phi).options(width=600)
def VisuFoncEchel(N,qmf,scale,pos):
    Lmax=pywt.dwt_max_level(N,pywt.Wavelet(qmf).dec_len)
    L=min(scale,Lmax)
    Ne=int(np.floor(N/2**L))
    k=int(np.floor(Ne*pos))
    temp=np.zeros(N)
    WT= pywt.wavedecn(temp, qmf, mode='per', level=L)
    arr, coeff_slices = pywt.coeffs_to_array(WT)
    temp[k]=1
    coeffs_from_arr = pywt.array_to_coeffs(temp, coeff_slices)
    Phi=pywt.waverecn(coeffs_from_arr,qmf,mode='per')
    return hv.Curve(Phi).options(width=600)

def VisuOnd(N,qmf,scale,pos):
    Lmax=pywt.dwt_max_level(N,pywt.Wavelet(qmf).dec_len)
    L=min(scale,Lmax)
    Ne=int(np.floor(N/2**L))
    k=int(np.floor(Ne*pos))
    temp=np.zeros(N)
    WT= pywt.wavedecn(temp, qmf, mode='per', level=L)
    arr, coeff_slices = pywt.coeffs_to_array(WT)
    temp[k+Ne]=1
    coeffs_from_arr = pywt.array_to_coeffs(temp, coeff_slices)
    Phi=pywt.waverecn(coeffs_from_arr,qmf,mode='per')
    return pn.Row(hv.Curve(Phi).options(width=600))
def ApproxLin(S,qmf,scale):
    # Approximation linéaire dans une base d'ondelettes. l'ondelette est spécifiée par la str qmf 
    # L'approximation est une projection sur l'espace V_j de dimension 2^scale. 
    N1=len(S)
    Lmax=pywt.dwt_max_level(len(S),pywt.Wavelet(qmf).dec_len)
    WT= pywt.wavedecn(S, qmf, mode='per', level=Lmax)
    arr, coeff_slices = pywt.coeffs_to_array(WT)
    temp=np.zeros(N1)
    temp[0:2**scale]=arr[0:2**scale]
    Wtemp = pywt.array_to_coeffs(temp, coeff_slices)
    Srec=pywt.waverecn(Wtemp,qmf,mode='per')
    barres=hv.VLine(N1/2-0.5).options(color='black',line_width=1)
    LN=int(np.floor(np.log2(N1)))
    for k in np.arange(LN-Lmax,LN):
        barres=barres*hv.VLine(2**k-0.5).options(color='black',line_width=1)
    return pn.Column(hv.Curve(S)*hv.Curve(Srec).options(color='red',width=600)\
                     ,hv.Curve(arr)*hv.Curve(temp).options(color='red',width=600)*barres)
def ApproxLinSimple(S,qmf,scale):
    # Approximation linéaire dans une base d'ondelettes. l'ondelette est spécifiée par la str qmf 
    # L'approximation est une projection sur l'espace V_j de dimension 2^scale. 
    N1=len(S)
    Lmax=pywt.dwt_max_level(len(S),pywt.Wavelet(qmf).dec_len)
    WT= pywt.wavedecn(S, qmf, mode='per', level=Lmax)
    arr, coeff_slices = pywt.coeffs_to_array(WT)
    temp=np.zeros(N1)
    temp[0:2**scale]=arr[0:2**scale]
    Wtemp = pywt.array_to_coeffs(temp, coeff_slices)
    Srec=pywt.waverecn(Wtemp,qmf,mode='per')
    return Srec

def ApproxOnd1D(S,qmf,L,T):
    Lmax=pywt.dwt_max_level(len(S),pywt.Wavelet(qmf).dec_len)
    L1=min(L,Lmax)
    WT= pywt.wavedecn(S, qmf, mode='per', level=L1)
    arr, coeff_slices = pywt.coeffs_to_array(WT)
    WTS=arr*(np.abs(arr)>T)
    coeffs_from_arr = pywt.array_to_coeffs(WTS, coeff_slices)
    Srec=pywt.waverecn(coeffs_from_arr,qmf,mode='per')
    return Srec
def ApproxOnd1Dv2(S,qmf,L,N):
    N1=len(S)
    Lmax=pywt.dwt_max_level(len(S),pywt.Wavelet(qmf).dec_len)
    L1=min(L,Lmax)
    WT= pywt.wavedecn(S, qmf, mode='per', level=L1)
    arr, coeff_slices = pywt.coeffs_to_array(WT)
    Ind=np.argsort(np.abs(arr))
    WTS=np.zeros(N1)
    WTS[Ind[N1-N:N1]]=arr[Ind[N1-N:N1]]
    coeffs_from_arr = pywt.array_to_coeffs(WTS, coeff_slices)
    Srec=pywt.waverecn(coeffs_from_arr,qmf,mode='per')
    return Srec
wavelist = ['haar','db2','db3','db4','coif1','coif2','coif3','sym2','sym3']
class VisuPhi(param.Parameterized):
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    scale = param.Integer(12,bounds=(0,15))
    pos = param.Number(0.5,bounds=(0,0.95))
    #T = param.Number(1,bounds=(1,100))
    #@param.depends('wave', 'T', 'L')
    def view(self):
        N=2**15
        Affich=VisuFoncEchel(N,self.wave,self.scale,self.pos)
        return Affich
class VisuPsi(param.Parameterized):
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    scale = param.Integer(12,bounds=(0,15))
    pos = param.Number(0.5,bounds=(0,0.95))
    #T = param.Number(1,bounds=(1,100))
    #@param.depends('wave', 'T', 'L')
    def view(self):
        N=2**15
        Affich=VisuOnd(N,self.wave,self.scale,self.pos)
        return Affich

Before giving examples of wavelets or wavelet transform, we can give the specific point of view of wavelets on signal and image processing. Where the Fourier analysis propose a decomposition of a signal into various frequences, the wavelet analysis decompose a signal into various scales. The wavelet analysis is a coarse to fine analysis. 
Functions are approximated at several scales and the wavelets coefficients contain the informations necessary to go from a coarse resulution to finer resolutions. There are several small wavelets associated to a given scale (or resolution).
Each wavelet basis is defined by a mother wavelet and all functions (actually nearly all) of the basis are images of this mother wavelet by translation and scaling. All functions in a basis have the same shape, just like all complex exponential have the same shape with various frequencies. 
Each wavelet $\psi_{j,k}$ in the basis is defined by two parameters (in 1D), one scale parameter denoted $j$ and one space parameter denoted $k$. Hence, each wavelet coefficient $\langle f,\psi_{j,k}\rangle$ corresponds to a "feature" or a "detail" at a given scale and at a given position. 

A Wavelet $\Psi$ is a function defined in $L^2(\mathbb{R})$ and a wavelet analysis consist on computing scalar products of a given function $f$ and functions $\psi_{j,k}$ which are scaled and translated versions of $\psi$. 

We will see that we can also define a wavelet analysis of vectors and images using a fast algorithm. 

Indeed, we can understand the philosophy of the discrete wavelet analysis of vectors and images without considering the continuous setting and in some way any technical mathematical tools at all via the so called "Lazy wavelet transform" :
https://en.wikipedia.org/wiki/Generalized_lifting

Let's suppose we have a vector $v\in\mathbb{R}^{1024}$ we want to compress, that is we want to propose an accurate approximation of $v$ with a small number of paramters. If the components of $v$ are redundant, it seems possible.

In [ ]:
hv.Curve(S3).opts(width=900)

A way to represent this vector into an efficient way is to provide a coarse version of $v$ that can be defined using only 512 coefficients and some detail informations using 512 coeffients that describe the difference between the original signal and this approximation (or coarse version). 

In [ ]:
Srec=ApproxLinSimple(S3,'haar',9)
hv.Curve(S3,label='Original Signal')*hv.Curve(Srec,label='Coarse approximation').opts(width=800)*hv.Curve(S3-Srec,label='Details')

A lazy wavelet scheme is a numerical scheme that divide a vector $v\in\mathbb{R}^{2n}$ into a "coarse approximation" that can be described with $n$ real number and a "detail vector" that can be described by $n$ real numbers. The details contains the information necesseray to go back from the coarse approximation to the original vector. If the coarse approximation is good, the details are equal or close to zero. Hence a Lazy wavelet scheme is a bijection from $\mathbb{R}^{2n}$ to $\mathbb{R}^n\times \mathbb{R}^{n}$ :


In [ ]:
temp=np.zeros(1024)
temp[0:512]=Srec[0:1024:2]
temp[512:1024]=S3[0:1024:2]-Srec[0:1024:2]
pn.Column(hv.Curve(S3).opts(width=900),hv.Curve(temp).opts(width=900))

More precisely, this numerical scheme is the first step of a lazy wavelet transform, since we can apply the same scheme to the coarse approximation, or the detail components or both. 

Actually, it can be shown that in many situations, there is no need to apply any transformation to the detail coefficients and in the lazy wavelet schemes a recursive transformation to the coarse approximation and details are left unchanged. Let's have a look of such schemes on images where the coarse image have 4 times less coefficients and for which the details can be devided into horizontal details, vertical details and diagonal details.

 

## Examples of 2D wavelet transform.

In [ ]:
#class PDF(object):
#  def __init__(self, pdf, size=(200,200)):
#    self.pdf = pdf
#    self.size = size
#
#  def _repr_html_(self):
#    return '<iframe src={0} width={1[0]} height={1[1]}></iframe>'.format(self.pdf, self.size)
#
#  def _repr_latex_(self):
#   return r'\includegraphics[width=1.0\textwidth]{{{0}}}'.format(self.pdf)

#PDF('Monument.pdf',size=(500,500))
Mo=chargeData('Monument').astype('uint8')
hv.RGB(Mo).opts(**options1)

After one step of a (lazy or not) wavelet transform we get this set of 4 images. The coarse approximation is on the top left, vertical details on the top right, horizontal details on the bottom left and diagonal details on the bottom right. From these 4 small images, we can recover the former. The advantage of this representation is that many coefficients are close to 0 which is better for compression, denoising and most image processing analysis.

In [ ]:
Mo2=chargeData('Monument2').astype('uint8')
hv.RGB(Mo2).opts(**options1)

We can now apply the same
scheme to the coarse image without changing the details. We get then the reprensatin above with 4 small images and 3 bigger ones. 

In [ ]:
Mo3=chargeData('Monument3').astype('uint8')
hv.RGB(Mo3).opts(**options1)

We can apply the scheme again and again...

In [ ]:
Mo4=chargeData('Monument4').astype('uint8')
hv.RGB(Mo4).opts(**options1)

To any such approximation scheme we can associate a recursive "lazy wavelet transform" and there are many ways to create such a coarse approximation of a vector or an image and consequently many possible lazy wavelet transforms. What are the difference between wavelet transform and lazy wavelet transform ? 
Wavelet transform are specific lazy wavelet transform, for which the coarse approximation and the detail coefficients are equal to scalar products of a the vectr with some wavelet and scalr functions. 

As an example, a simple way to provide a coarse approximation of a vector $v\in\mathbb{R}^{2n}$ is to approximate each pair of component by the mean of the pair. The details are then the difference of the first value to the mean. This simple scheme is a lazy wavelet scheme... and is also the Haar wavelet transform in the discrete setting.  

We will see that this simple scheme is adapated to piecewise constant functions (and vectors) but less efficient to deal with more complexe data. To deal with piecewise regularity of higher orders (piecewise lienar, quadratic or more), we will need to use better approximations schemes leading to other wavelets.


Wavelet transform can be used for compression, but also for many other applications, such like denoising.

In [ ]:
Croco1=chargeData('Croco1').astype('uint8')
hv.RGB(Croco1).opts(**options2)

Here, we can threshold the small coefficients, that is setting small ones to 0, considering that these small coefficients are more likely to be "noise" than to belong to the "signal".

In [ ]:
Croco2=chargeData('Croco2').astype('uint8')
hv.RGB(Croco2).opts(**options2)

We will aslo see how the wavelet parcimony (the fact that there is a small number of active detail coefficients) can be used to remove a blur from an image :

In [ ]:
plane=chargeData('avion').astype('uint8')
hv.RGB(plane).opts(**options1)

In [ ]:
plane2=chargeData('avion2').astype('uint8')
hv.RGB(plane2).opts(**options1)

Or to recover lost pixels of a natural image.

In [ ]:
Lenna1=chargeData('LennaMasked').astype('uint8')
hv.RGB(Lenna1).opts(**options1)

In [ ]:
Lenna2=chargeData('LennaInpainted').astype('uint8')
hv.RGB(Lenna2).opts(**options1)

## 1D Wavelet basis. Wavelet functions and scale functions. 

In the two next dashboards, you can have a look on several scale functions and wavelet functions we will study during the semester. We will first study the Haar transform.

In [ ]:
Visupsi = VisuPsi()
pn.Row(Visupsi.param,Visupsi.view)

In [ ]:
Visuphi = VisuPhi()
pn.Row(Visuphi.param,Visuphi.view)

### Reproducing Formula.

An important formula, on which is based the Fast Wavelet Transform is the following $\forall (j,k)$ :
\begin{equation}
\phi_{j,k}=\sum_{n\in\mathbb{Z}}h_n\phi_{j-1,n+2k}
\end{equation}
which gives the link between a scale function and scales functions a scale above.

This identity can be simplefied for the Haar wavelet into 
\begin{equation}
\phi_{j,k}=\frac{\sqrt{2}}{2}\phi_{j-1,2k}+\frac{\sqrt{2}}{2}\phi_{j-1,2k+1}
\end{equation}


In [ ]:
class RF(param.Parameterized):
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    scale = param.Integer(6,bounds=(0,9))
    k = param.Integer(9,bounds=(1,31))
    def view(self):
        w=FoncEchel(1024,self.wave,1,8)
        coefs=w[np.abs(w)>0]
        nb=len(coefs)
        phi1=FoncEchel(1024,self.wave,self.scale,self.k) 
        phi=hv.Curve(FoncEchel(1024,self.wave,self.scale-1,2*self.k-int(nb/2)+1)*coefs[0])
        for ind in np.arange(1,nb):
            phi=phi*hv.Curve(FoncEchel(1024,self.wave,self.scale-1,2*self.k+ind-int(nb/2)+1)*coefs[ind])
            #phi3=(-1)*FoncEchel(1024,'haar',self.scale-1,2*self.k+1)/np.sqrt(2)
        return pn.Column(hv.Curve(phi1).options(color='red',width=600),phi.options(width=600))
    

In [ ]:
rf = RF()
pn.Row(rf.param,rf.view)

Similarly there is a equation linking the wavelet to the scale functions, one scale above also available
$\forall (j,k)$ :
\begin{equation}
\psi_{j,k}=\sum_{n\in\mathbb{Z}}(-1)^n(h_{1-n}\phi_{j-1,n+2k}
\end{equation}
This identity can be simplefied for the Haar wavelet into 
\begin{equation}
\psi_{j,k}=-\frac{\sqrt{2}}{2}\phi_{j-1,2k}+\frac{\sqrt{2}}{2}\phi_{j-1,2k+1}
\end{equation}

Remark : The choice of coefficients $-\frac{\sqrt{2}}{2}$ and $\frac{\sqrt{2}}{2}$ is actualy a convention, you may find in books $\frac{\sqrt{2}}{2}$ and $-\frac{\sqrt{2}}{2}$. But most of the time it is pointless since the choices are made accordingly in the direct and the inverse transform. All wavelets can be reversed in time without any damages.

In [ ]:
class RF2(param.Parameterized):
    wave = param.ObjectSelector(default="db2",objects=wavelist)
    scale = param.Integer(7,bounds=(0,9))
    k = param.Integer(4,bounds=(1,31))
    def view(self):
        w=Ond(1024,self.wave,1,8)
        coefs=w[np.abs(w)>0]
        nb=len(coefs)
        phi1=Ond(1024,self.wave,self.scale,self.k) 
        phi=hv.Curve(FoncEchel(1024,self.wave,self.scale-1,2*self.k-int(nb/2)+1)*coefs[0])
        for ind in np.arange(1,nb):
            phi=phi*hv.Curve(FoncEchel(1024,self.wave,self.scale-1,2*self.k+ind-int(nb/2)+1)*coefs[ind])
        return pn.Column(hv.Curve(phi1).options(color='red',width=600),phi.options(width=600))

In [ ]:
rf2 = RF2()
pn.Row(rf2.param,rf2.view)

In [ ]:
ramp=np.arange(0,1024)/1024
def fonc1(t):
    if t<1/3:
        y=t**2
    elif t<2/3:
        y=t/5
    else:
        y=t*(1-t)
    return 10*y            
vfonc1=np.vectorize(fonc1)
f1=vfonc1(ramp)
def fonc2(t):
    if t<1/3:
        y=t**3
    elif t<2/3:
        y=-0.1-t**2/2+2*t/3
    else:
        y=t*(1-t)*t
    return 30*y 
vfonc2=np.vectorize(fonc2)
vfonc1=np.vectorize(fonc1)
f1=vfonc1(ramp)
f2=vfonc2(ramp)
signauxRef= {"Piece" : S3*2,"Blocks" : S4*2,"Ramp":2*ramp,"Para" : f1, "Cubic":f2}


In [ ]:
options = dict(width=800,height=150,toolbar=None,xaxis=None,yaxis=None)
options2 = dict(width=600,height=250,toolbar=None,xaxis=None,yaxis=None)

def VisuOnd2(N,qmf,scale,k):
    Lmax=pywt.dwt_max_level(N,pywt.Wavelet(qmf).dec_len)
    L=min(scale,Lmax)
    Ne=int(np.floor(N/2**L))
    temp=np.zeros(N)
    WT= pywt.wavedecn(temp, qmf, mode='per', level=L)
    arr, coeff_slices = pywt.coeffs_to_array(WT)
    temp[k+Ne]=1
    coeffs_from_arr = pywt.array_to_coeffs(temp, coeff_slices)
    Psi=pywt.waverecn(coeffs_from_arr,qmf,mode='per')*2**(scale/2)
    return Psi
class Visu(param.Parameterized):
    sig = param.ObjectSelector(default="Para",objects=signauxRef.keys())
    wave = param.ObjectSelector(default="db2",objects=wavelist)
    scale= param.Integer(6,bounds=(4,6))
    k= param.Integer(7,bounds=(0,64))
    def view(self):
        k2=min(self.k,2**(10-self.scale))
        s=signauxRef[self.sig]
        N=2**10
        t=np.arange(0,N)/N
        scale=self.scale  
        Psi=VisuOnd2(N,self.wave,self.scale,k2)
        Affich=hv.Curve(Psi).opts(**options2,color='red')
        I=np.where(np.abs(Psi)>0)
        s2=0*s
        s2[I]=s[I]
        WT= pywt.wavedec(s,self.wave, mode='per', level=7)
        coefs=WT[7-scale+1]
        coefs2=0*coefs
        coefs2[k2]=coefs[k2]
        strscale="%2.0f" % scale
        return pn.Column(hv.Curve(s).opts(**options2,title='Signal étudié')*hv.Area(s2).opts(color='red')*Affich.opts(color='black')\
                                          ,hv.Bars(coefs).opts(**options2,title='Coefficients sur W-'+strscale)\
                         *hv.Bars(coefs2).opts(color='red'))

The next dashboard illustrate the part of the function "analysed" by a given wavelet. The red bar on the bottom corresponds to the associated coefficient in the wavelet transform.

In [ ]:
vis= Visu()
pn.Row(vis.param,vis.view)

## Approximation in the wavelet domain.

# Linear wavelet approximation, projection on $V_j$ spaces.

In [ ]:
class WaveApproxLin(param.Parameterized):
    signal = param.ObjectSelector(default="Blocks",objects=signauxRef.keys())
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    scale = param.Integer(7,bounds=(0,10))
    #T = param.Number(1,bounds=(1,100))
    #@param.depends('wave', 'T', 'L')
    def view(self):
        s=signauxRef[self.signal]
        Affich=ApproxLin(s,self.wave,self.scale)
        return Affich

In [ ]:
waveapproxlin = WaveApproxLin()
pn.Row(waveapproxlin.param,waveapproxlin.view)

### Non linear wavelet approximation.

In [ ]:
def ApproxNonLin(S,N):
    Nf=np.size(S)
    FS=fftpack.fft(S)
    AFS=-np.abs(FS)
    I=np.argsort(AFS)
    temp=np.zeros(Nf,dtype=complex)
    temp[I[0:N]]=FS[I[0:N]]
    print(np.real(temp[I[1]]))
    print(np.real(FS[I[1]]))    
    y=np.real(fftpack.ifft(temp))
    #plt.plot(np.abs(temp))
    #plt.show
    return y

In [ ]:
def ApproxOnd1Dv2(S,qmf,L,N):
    N1=len(S)
    Lmax=pywt.dwt_max_level(len(S),pywt.Wavelet(qmf).dec_len)
    L1=min(L,Lmax)
    WT= pywt.wavedecn(S, qmf, mode='per', level=L1)
    arr, coeff_slices = pywt.coeffs_to_array(WT)
    Ind=np.argsort(np.abs(arr))
    WTS=np.zeros(N1)
    WTS[Ind[N1-N:N1]]=arr[Ind[N1-N:N1]]
    coeffs_from_arr = pywt.array_to_coeffs(WTS, coeff_slices)
    Srec=pywt.waverecn(coeffs_from_arr,qmf,mode='per')
    return Srec

In [ ]:
class WaveExplorerv2(param.Parameterized):
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    signal = param.ObjectSelector(default="Blocks",objects=signauxRef.keys())
    L = param.Integer(9,bounds=(0,9))
    N = param.Integer(25,bounds=(1,1023))
  #  @param.depends('wave', 'N', 'L')
    def view(self):
        s=signauxRef[self.signal]
        Srec = ApproxOnd1Dv2(s,self.wave,self.L,self.N)
        return hv.Curve(s).options(color='blue',width=600)*hv.Curve(Srec).options(color='red',width=600)

In [ ]:
waveexplo2 = WaveExplorerv2()
pn.Row(waveexplo2.param,waveexplo2.view)